<a href="https://colab.research.google.com/github/AMerrington/sense-hackathon/blob/leopauly/SENSE_CDT_Practical_Session_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automated Sentinel-1 Ice, Water, Land Segmentation Challenge

Version : leopauly

In [1]:
## Imports
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import torch
import torch.nn as nn
from torch import optim
import numpy as np


In [2]:
## Mounting drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [3]:
## Setting variables
SAMPLING_DIR ='/content/drive/MyDrive/png_samples_200x200/'
META_DIR ='/content/drive/MyDrive/tiff_sample_metadata/meta3.npy'
TRAIN_SIZE = 0.7
VALID_SIZE = 1.0 - TRAIN_SIZE
LABELS = {"L": 0,"W": 1,"I": 2,}

In [44]:
## Checking loaded dataset

print('Metafile details:')
meta=np.load(META_DIR).transpose()
print('Meta file size:',np.shape(meta))
print('labels:',meta[3])
print('x_cord:',meta[2])
print('y_cord:',meta[1])
print('parent tile name:',meta[0])

Metafile details:
Meta file size: (4, 2513)
labels: ['W' 'W' 'W' ... 'I' 'I' 'L']
x_cord: ['5113' '8524' '12549' ... '4317' '2531' '3529']
y_cord: ['7948' '6878' '5053' ... '5365' '8466' '4588']
parent tile name: ['7520' '7521' '7522' ... '10030' '10031' '10032']


In [49]:
## Database class
class PolarPatch(Dataset):
    def __init__(self, transform=None, split="train"):
        super(PolarPatch, self).__init__()

        assert split in ["train", "val"]
        
        meta = np.load('/content/drive/MyDrive/tiff_sample_metadata/meta3.npy').transpose()

        train_dim = int(TRAIN_SIZE * len(meta))
        
        if split == "train":
            meta = meta[:train_dim]
        else:
            meta = meta[train_dim:]                   

        self.images = range(len(meta))
        self.coords = [meta[1],meta[2]]#[(row[1], row[2]) for row in meta]

        # Targets in integer form for computing cross entropy
        self.targets = meta[3]#[LABELS[row[3]] for row in meta]
        self.transform = transform


    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):

        print(index)

        x = Image.open(SAMPLING_DIR + str(self.images[index]) + ".png") # change this file format if needed
        y = self.targets[index]
        coord = self.coords[index]

        if self.transform:
        	x = self.transform(x)

        return x, y, coord

In [50]:
## Data pre-processing (todo: normalisation)
data_transform = transforms.Compose([transforms.ToTensor()])

In [51]:
## Dataloaders
BATCH_SIZE = 128 #todo: set this

train_set = PolarPatch(split='train',transform=data_transform)
#test_set = PolarPatch(split='test',transform=data_transform)

train_loader = torch.utils.data.DataLoader(train_set,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)

IndexError: ignored

In [ ]:
## Model
class PolarNet(nn.Module):
    def __init__(self, n_classes=3):
        super(PolarNet, self).__init__()

        self.features = nn.Sequential(
            # TODO: build your own architecture here; one conv layer and ReLU here as an example only
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True), 
        )

        self.classifier = nn.Sequential(
            # TODO: continue classifier section of architecture here for classification approach;
            # otherwise, remove and add in upscaling for a fully-convolutional segmentation approach 
            nn.Linear(4096, n_classes),
        )      

    def forward(self, x):
        # as an example; alter as needed depending on your architecture
        x = self.features(x)

        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
## Training
# Device configuration - defaults to CPU unless GPU is available on device
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = PolarNet().to(DEVICE)
criterion = nn.CrossEntropyLoss()

# Stochastic gradient descent - TODO: alter as needed
optimizer = optim.SGD(model.parameters(),lr=0.001,weight_decay=0.0005,momentum=0.9)

In [65]:
iterations=2
for epoch in range(iterations): 
    running_loss = 0.0

    for i, data in enumerate(train_loader, 0):
        #Loading data. todo: use dataloaders 
        inputs, labels = data

        # Forward pass+backward pass
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

FileNotFoundError: ignored

In [15]:
## Evaluation


In [38]:
## Misc #Remove before presentation




print(range(len(meta)))

Meta_data size: (4, 2513)
labels: ['W' 'W' 'W' ... 'I' 'I' 'L']
x_cord: ['5113' '8524' '12549' ... '4317' '2531' '3529']
y_cord: ['7948' '6878' '5053' ... '5365' '8466' '4588']
name: ['7520' '7521' '7522' ... '10030' '10031' '10032']
range(0, 4)


In [34]:
print(meta[3])

['W' 'W' 'W' ... 'I' 'I' 'L']
